<a href="https://colab.research.google.com/github/frank-morales2020/Cloud_curious/blob/master/FT_V2TXT_DEMO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 540.5/540.5 kB 39.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# 1. INSTALL REQUIRED DEPENDENCIES
!pip install -q mistral-common

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 138.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.3/74.3 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 160.9 MB/s eta 0:00:00


In [4]:
!pip install --upgrade transformers -q
!pip install -U bitsandbytes>=0.46.1 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 141.3 MB/s eta 0:00:00


In [ ]:
import os
os.kill(os.getpid(), 9)

Original duration: 936.28s for barackobama2004dncARXE.mp3

Original duration: 210.68s for barackobamatransitionaddress1.mp3

Original duration: 183.02s for brad_pitt_sag_2020.mp3

Original duration: 2415.33s for mandela_davos_1999.mp3

Original duration: 1801.37s for mark_carney_davos_2026.mp3

Original duration: 2585.63s for mlk_mountaintop_1968.mp3

In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
# SUPPRESS WARNINGS - Add this at the very top
import warnings
warnings.filterwarnings("ignore", message="Some matrices hidden dimension is not a multiple of 64")
warnings.filterwarnings("ignore", module="bitsandbytes")

import os
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import librosa
from datasets import Dataset
from dataclasses import dataclass
from transformers import (
    AutoProcessor,
    AutoModelForSpeechSeq2Seq,
    BitsAndBytesConfig,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import transformers.models.voxtral_realtime.modeling_voxtral_realtime as vox_mod

from warnings import simplefilter
simplefilter(action="ignore", category=FutureWarning)

# ==================== CONFIG ====================
MODEL_ID = "mistralai/Voxtral-Mini-4B-Realtime-2602"
OUTPUT_DIR = "/content/drive/MyDrive/data/H2E_Challenge/Voxtral_FineTune"
AUDIT_PATH = "/content/drive/MyDrive/data/H2E_Challenge/H2E_Final_Performance_Audit.csv"
TARGET_SR = 16000
CHUNK_LENGTH_SEC = 30.0
MAX_TEXT_LENGTH = 448

# ==================== 1. DATASET PREPARATION ====================
df = pd.read_csv(AUDIT_PATH)
paths = [f"/content/drive/MyDrive/data/{f}" for f in df['File']]
texts = df['Transcript'].astype(str).tolist()

chunked_audios, chunked_texts = [], []
for path, text in zip(paths, texts):
    try:
        array, _ = librosa.load(path, sr=TARGET_SR, mono=True)
        max_samples = int(CHUNK_LENGTH_SEC * TARGET_SR)
        if len(array) > max_samples: array = array[:max_samples]
        chunked_audios.append({"array": array.astype(np.float32).tolist(), "sampling_rate": TARGET_SR})
        chunked_texts.append(text)
    except Exception as e: print(f"✗ Failed {path}: {e}")

dataset = Dataset.from_dict({"audio": chunked_audios, "text": chunked_texts})

# ==================== 2. MODEL SETUP ====================
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"
)

# Get model dimensions
lm_hidden_size = model.config.hidden_size
print(f"Language model hidden size: {lm_hidden_size}")

# Audio feature dimension for Voxtral
audio_feature_dim = 1280
print(f"Audio feature dimension: {audio_feature_dim}")
print(f"Setting up adapter: {audio_feature_dim} → {lm_hidden_size}")

# Create adapter that projects audio features to match language model hidden size
class AudioFeatureAdapter(nn.Module):
    def __init__(self, in_dim, out_dim):
        super().__init__()
        self.proj = nn.Linear(in_dim, out_dim)
    def forward(self, x):
        return self.proj(x)

# Initialize adapter with correct dimensions
model.audio_adapter = AudioFeatureAdapter(audio_feature_dim, lm_hidden_size).to(model.device).to(torch.bfloat16)

def preprocess_function(examples):
    audio_arrays = []
    for x in examples["audio"]:
        arr = np.array(x["array"], dtype=np.float32)
        audio_arrays.append(arr)

    audio_inputs = processor.feature_extractor(
        audio_arrays,
        sampling_rate=TARGET_SR, return_tensors="np", padding=True
    )

    input_features = torch.tensor(audio_inputs["input_features"], dtype=torch.bfloat16)

    text_inputs = processor.tokenizer(
        examples["text"], return_tensors="np", padding="max_length",
        truncation=True, max_length=MAX_TEXT_LENGTH
    )

    return {
        "input_features": input_features,
        "input_ids": torch.tensor(text_inputs["input_ids"], dtype=torch.long),
        "labels": torch.tensor(text_inputs["input_ids"], dtype=torch.long),
        "attention_mask": torch.tensor(text_inputs["attention_mask"], dtype=torch.long)
    }

dataset = dataset.map(preprocess_function, batched=True, batch_size=2, remove_columns=dataset.column_names)
dataset.set_format("torch")

# Prepare for k-bit training but disable gradient checkpointing
model = prepare_model_for_kbit_training(model)
model.gradient_checkpointing_disable()  # Explicitly disable gradient checkpointing

model = get_peft_model(model, LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    task_type="CAUSAL_LM"
))

# ==================== 3. SIMPLIFIED PATCHED FORWARD ====================
def patched_forward(
    self, input_ids=None, input_features=None, attention_mask=None, position_ids=None,
    past_key_values=None, encoder_past_key_values=None, padding_cache=None,
    inputs_embeds=None, encoder_inputs_embeds=None, labels=None, use_cache=None,
    cache_position=None, logits_to_keep=0, num_delay_tokens=None, **kwargs
):
    # Get text embeddings
    if inputs_embeds is None:
        inputs_embeds = self.get_input_embeddings()(input_ids)

    batch_size = inputs_embeds.shape[0]
    device = inputs_embeds.device

    # Process audio if provided
    if input_features is not None:
        # Get audio features
        audio_outputs = self.audio_tower(
            input_features.to(device, dtype=torch.bfloat16),
            output_hidden_states=True,
            return_dict=True
        )

        # Project audio features
        audio_features = audio_outputs.last_hidden_state
        audio_projected = self.audio_adapter(audio_features)

        # Simple concatenation
        inputs_embeds = torch.cat([audio_projected, inputs_embeds], dim=1)

        # Update attention mask
        if attention_mask is not None:
            audio_mask = torch.ones(batch_size, audio_projected.shape[1],
                                   device=device, dtype=attention_mask.dtype)
            attention_mask = torch.cat([audio_mask, attention_mask], dim=1)

    # Time conditioning - FIXED: properly handle dimensions
    if num_delay_tokens is None:
        num_delay_tokens = self.config.default_num_delay_tokens

    # Create time tensor with correct shape for t_cond
    time_tensor = torch.full((batch_size,), num_delay_tokens, device=device, dtype=torch.long)
    t_cond = self.time_embedding(time_tensor)  # Shape: [batch_size, hidden_size]

    # Call language model
    lm_outputs = self.language_model(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        use_cache=False,  # Disable cache to avoid issues
        t_cond=t_cond,  # Pass t_cond directly
        **kwargs
    )

    logits = lm_outputs.logits
    loss = None

    if labels is not None:
        # Create labels that ignore audio tokens
        if input_features is not None:
            audio_len = audio_projected.shape[1]
            new_labels = torch.full((batch_size, audio_len + labels.shape[1]), -100,
                                   device=device, dtype=labels.dtype)
            new_labels[:, audio_len:] = labels
            labels = new_labels

        # Shift for next token prediction
        shift_logits = logits[..., :-1, :].contiguous()
        shift_labels = labels[..., 1:].contiguous()
        loss = nn.functional.cross_entropy(
            shift_logits.view(-1, shift_logits.size(-1)),
            shift_labels.view(-1),
            ignore_index=-100
        )

    return vox_mod.VoxtralRealtimeCausalLMOutputWithPast(
        loss=loss,
        logits=logits,
        past_key_values=lm_outputs.past_key_values
    )

# Apply the patch
vox_mod.VoxtralRealtimeForConditionalGeneration.forward = patched_forward

# ==================== 4. TRAINING ====================
@dataclass
class SimpleCollator:
    def __call__(self, features):
        batch = {}
        for key in ["input_features", "input_ids", "labels", "attention_mask"]:
            if key in features[0]:
                tensors = [f[key] for f in features]
                if key == "input_features":
                    batch[key] = torch.stack(tensors).to(torch.bfloat16)
                else:
                    batch[key] = torch.stack(tensors)
        return batch

# Disable all caching and checkpointing in training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate=1e-4,
    max_steps=500,
    bf16=True,
    logging_steps=10,
    report_to="none",
    remove_unused_columns=False,
    save_strategy="no",
    dataloader_drop_last=False,
    gradient_checkpointing=False,  # Explicitly disable
    use_cache=False,  # Disable cache
    ddp_find_unused_parameters=False,
    prediction_loss_only=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    data_collator=SimpleCollator(),
)

print("🚀 Starting Training with Fixed Forward Pass...")
print(f"Model device: {model.device}")
print(f"Hidden size: {lm_hidden_size}")
trainer.train()


print("\n" + "="*50)
print("💾 Saving fine-tuned model...")

# Create the adapter directory
final_adapter_path = os.path.join(OUTPUT_DIR, "final_adapter")
os.makedirs(final_adapter_path, exist_ok=True)

# Save the model and processor
model.save_pretrained(final_adapter_path)
processor.save_pretrained(final_adapter_path)

print(f"✅ Model saved to: {final_adapter_path}")

# Verify the save
print("\n📁 Saved files:")
if os.path.exists(final_adapter_path):
    for file in os.listdir(final_adapter_path):
        file_path = os.path.join(final_adapter_path, file)
        size = os.path.getsize(file_path) / 1024  # Size in KB
        print(f"  - {file} ({size:.2f} KB)")
else:
    print("❌ Save failed - directory not found!")

print("\n🎉 Training complete! You can now use the model for inference.")

Loading weights:   0%|          | 0/711 [00:00<?, ?it/s]

Language model hidden size: 3072
Audio feature dimension: 1280
Setting up adapter: 1280 → 3072


Map:   0%|          | 0/6 [00:00<?, ? examples/s]

🚀 Starting Training with Fixed Forward Pass...
Model device: cuda:0
Hidden size: 3072


Step,Training Loss
10,41.992667
20,27.299902


## INFERENCE-CODE

In [4]:
# inference_fixed.py - Corrected version with proper function ordering

import torch
import torch.nn as nn
import librosa
import numpy as np
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
from peft import PeftModel
import os

# ==================== CONFIG ====================
MODEL_ID = "mistralai/Voxtral-Mini-4B-Realtime-2602"
OUTPUT_DIR = "/content/drive/MyDrive/data/H2E_Challenge/Voxtral_FineTune"
ADAPTER_PATH = os.path.join(OUTPUT_DIR, "final_adapter")
DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"
TARGET_SR = 16000

# ==================== LOAD MODEL ====================
print("🚀 Loading Voxtral Model...")
print(f"Adapter path: {ADAPTER_PATH}")
print(f"Device: {DEVICE}")
print("="*50)

# Load processor
processor = AutoProcessor.from_pretrained(MODEL_ID, trust_remote_code=True)

# Load base model
print("\n📦 Loading base model...")
model = AutoModelForSpeechSeq2Seq.from_pretrained(
    MODEL_ID,
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.bfloat16
)

# Load your fine-tuned adapter
print(f"\n🔧 Loading fine-tuned adapter from: {ADAPTER_PATH}")
model = PeftModel.from_pretrained(model, ADAPTER_PATH)
model.eval()
print("✓ Adapter loaded successfully!")

# ==================== AUDIO ADAPTER DEFINITION ====================
class AudioFeatureAdapter(nn.Module):
    def __init__(self, in_dim=1280, out_dim=3072):
        super().__init__()
        self.proj = nn.Linear(in_dim, out_dim)
    def forward(self, x):
        return self.proj(x)

# Add adapter to model
if not hasattr(model, 'audio_adapter'):
    model.audio_adapter = AudioFeatureAdapter().to(model.device).to(torch.bfloat16)
    print("✓ Audio adapter added to model")

# ==================== FIXED PATCHED FORWARD FOR INFERENCE ====================
import transformers.models.voxtral_realtime.modeling_voxtral_realtime as vox_mod

# First, save the original prepare_inputs_for_generation method
original_prepare_inputs = vox_mod.VoxtralRealtimeForConditionalGeneration.prepare_inputs_for_generation

# Define patched functions BEFORE applying them
def patched_prepare_inputs_for_generation(self, input_ids, **kwargs):
    # If we have input_features, we don't need input_ids
    if kwargs.get('input_features') is not None:
        # Create a dummy input_ids to satisfy the method signature
        # but it won't be used because we'll override with inputs_embeds
        dummy_input_ids = torch.ones((input_ids.shape[0], 1), dtype=torch.long, device=input_ids.device)
        kwargs['inputs_embeds'] = None  # Will be created in forward
        return {
            'input_ids': dummy_input_ids,
            'input_features': kwargs.get('input_features'),
            'attention_mask': kwargs.get('attention_mask'),
            'use_cache': kwargs.get('use_cache', True),
        }
    return original_prepare_inputs(self, input_ids, **kwargs)

def patched_forward(
    self, input_ids=None, input_features=None, attention_mask=None, position_ids=None,
    past_key_values=None, encoder_past_key_values=None, padding_cache=None,
    inputs_embeds=None, encoder_inputs_embeds=None, labels=None, use_cache=None,
    cache_position=None, logits_to_keep=0, num_delay_tokens=None, **kwargs
):
    # Process audio features
    if input_features is not None:
        # Get audio features from audio_tower
        audio_outputs = self.audio_tower(
            input_features.to(self.device, dtype=torch.bfloat16),
            output_hidden_states=True,
            return_dict=True
        )

        # Project audio features
        audio_features = audio_outputs.last_hidden_state
        inputs_embeds = self.audio_adapter(audio_features)

        # Create attention mask for audio
        batch_size = inputs_embeds.shape[0]
        if attention_mask is None:
            attention_mask = torch.ones(batch_size, inputs_embeds.shape[1],
                                       device=self.device, dtype=torch.long)
    else:
        # Fallback to text-only mode
        if inputs_embeds is None and input_ids is not None:
            inputs_embeds = self.get_input_embeddings()(input_ids)

    # Time conditioning
    if num_delay_tokens is None:
        num_delay_tokens = self.config.default_num_delay_tokens

    batch_size = inputs_embeds.shape[0] if inputs_embeds is not None else input_ids.shape[0]
    time_tensor = torch.full((batch_size,), num_delay_tokens, device=self.device, dtype=torch.long)
    t_cond = self.time_embedding(time_tensor)

    # Call language model with inputs_embeds
    lm_outputs = self.language_model(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        position_ids=position_ids,
        past_key_values=past_key_values,
        use_cache=use_cache if use_cache is not None else True,
        t_cond=t_cond,
        **kwargs
    )

    logits = lm_outputs.logits

    return vox_mod.VoxtralRealtimeCausalLMOutputWithPast(
        loss=None,
        logits=logits,
        past_key_values=lm_outputs.past_key_values
    )

# Apply the patches
vox_mod.VoxtralRealtimeForConditionalGeneration.forward = patched_forward
vox_mod.VoxtralRealtimeForConditionalGeneration.prepare_inputs_for_generation = patched_prepare_inputs
print("✓ Forward patches applied")

# ==================== INFERENCE FUNCTIONS ====================

def transcribe_audio(audio_path, max_new_tokens=200):
    """
    Transcribe a single audio file
    """
    try:
        print(f"\n📂 Processing: {os.path.basename(audio_path)}")

        # Load audio
        audio, _ = librosa.load(audio_path, sr=TARGET_SR, mono=True)

        # Prepare inputs - only audio features, no text
        inputs = processor(
            audio,
            sampling_rate=TARGET_SR,
            return_tensors="pt"
        )

        # Move to device
        input_features = inputs["input_features"].to(DEVICE).to(torch.bfloat16)

        # Generate - only pass input_features
        print("⚙️ Generating transcription...")
        with torch.no_grad():
            generated_ids = model.generate(
                input_features=input_features,
                max_new_tokens=max_new_tokens,
                do_sample=False,
                num_beams=1,
            )

        # Decode
        transcription = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

        # Print first 100 chars or full if shorter
        preview = transcription[:100] + "..." if len(transcription) > 100 else transcription
        print(f"📝 Transcription: {preview}")
        return transcription

    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()
        return None

def save_transcription(transcription, audio_path):
    """
    Save transcription to a text file
    """
    transcripts_dir = os.path.join(OUTPUT_DIR, "transcripts")
    os.makedirs(transcripts_dir, exist_ok=True)

    base_name = os.path.basename(audio_path).split('.')[0]
    output_path = os.path.join(transcripts_dir, f"{base_name}_transcript.txt")

    with open(output_path, 'w') as f:
        f.write(transcription)

    print(f"💾 Saved to: {output_path}")
    return output_path

def batch_transcribe(audio_files):
    """
    Transcribe multiple audio files
    """
    results = {}
    successful = 0
    failed = 0

    for i, audio_path in enumerate(audio_files):
        print(f"\n[{i+1}/{len(audio_files)}] ", end="")
        transcription = transcribe_audio(audio_path)
        if transcription:
            results[audio_path] = transcription
            save_transcription(transcription, audio_path)
            successful += 1
        else:
            failed += 1
        print("-" * 50)

    print(f"\n✅ Complete: {successful} successful, {failed} failed")
    return results

# ==================== TEST THE MODEL ====================
print("\n" + "="*50)
print("🎯 Testing the model on your audio files")
print("="*50)

# Get all audio files
data_dir = "/content/drive/MyDrive/data"
if os.path.exists(data_dir):
    audio_files = [os.path.join(data_dir, f) for f in os.listdir(data_dir)
                  if f.endswith(('.mp3', '.wav', '.m4a', '.flac'))]

    if audio_files:
        print(f"\nFound {len(audio_files)} audio files")

        # Test first file only first to verify it works
        print("\n" + "="*50)
        print("🔬 Testing first file to verify fix...")
        print("="*50)

        test_file = audio_files[0]
        print(f"Test file: {os.path.basename(test_file)}")
        transcription = transcribe_audio(test_file)

        if transcription:
            print("\n✅ First file succeeded! Saving result...")
            save_transcription(transcription, test_file)

            # Now ask user what to do
            print("\n" + "="*50)
            print("Select option:")
            print("1. Transcribe all remaining files")
            print("2. Transcribe a specific file")
            print("3. Exit")
            print("="*50)

            choice = input("\nEnter choice (1-3): ").strip()

            if choice == "1":
                print("\n🔄 Transcribing remaining files...")
                if len(audio_files) > 1:
                    batch_transcribe(audio_files[1:])
                else:
                    print("No remaining files to transcribe.")

            elif choice == "2":
                print("\nSelect file number:")
                for i, file in enumerate(audio_files):
                    status = "✓" if i == 0 else " "
                    print(f"  {status} {i+1}. {os.path.basename(file)}")

                try:
                    file_num = int(input("\nEnter file number: ")) - 1
                    if 0 <= file_num < len(audio_files):
                        if file_num == 0:
                            print("File 1 already transcribed. Transcribing again...")
                        transcription = transcribe_audio(audio_files[file_num])
                        if transcription:
                            save = input("\nSave transcription? (y/n): ").lower()
                            if save == 'y':
                                save_transcription(transcription, audio_files[file_num])
                    else:
                        print("❌ Invalid file number")
                except ValueError:
                    print("❌ Please enter a valid number")
        else:
            print("\n❌ First file failed. Error details above.")

            # Offer to try a different file
            print("\nTry a different file? (Enter file number or 'n' to exit)")
            for i, file in enumerate(audio_files):
                print(f"  {i+1}. {os.path.basename(file)}")

            try:
                file_num = int(input("\nEnter file number: ")) - 1
                if 0 <= file_num < len(audio_files):
                    transcription = transcribe_audio(audio_files[file_num])
                    if transcription:
                        save = input("\nSave transcription? (y/n): ").lower()
                        if save == 'y':
                            save_transcription(transcription, audio_files[file_num])
            except ValueError:
                print("Exiting...")
    else:
        print(f"No audio files found in {data_dir}")
else:
    print(f"Data directory not found: {data_dir}")

print("\n" + "="*50)
print("✅ Inference complete!")
print("="*50)

# ==================== SIMPLE FUNCTION FOR QUICK USE ====================
def quick_transcribe(audio_path):
    """
    Quick one-liner transcription function
    Usage: text = quick_transcribe('path/to/audio.mp3')
    """
    return transcribe_audio(audio_path)

print("\n📌 Quick transcription function available:")
print("   text = quick_transcribe('path/to/audio.mp3')")
print("\n📂 Transcripts will be saved to: {}/transcripts/".format(OUTPUT_DIR))